In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)

In [2]:
df = pd.DataFrame(pd.read_csv('iheart_projects - iheart_projects.csv'))
revenue = pd.DataFrame(pd.read_csv('report1619705426780.csv'))
revenue2 = pd.DataFrame(pd.read_csv('report1617632400435 - report1617632400435.csv'))

In [3]:
revenue['Form Date'] = pd.to_datetime(revenue['Form Date'], errors='coerce').dt.strftime('%Y%m%d')
revenue['Opportunity: Created Date'] = pd.to_datetime(revenue['Opportunity: Created Date'], errors='coerce').dt.strftime('%Y%m%d')
revenue['Opportunity: Close Date'] = pd.to_datetime(revenue['Opportunity: Close Date'], errors='coerce').dt.strftime('%Y%m%d')
# revenue = revenue.loc[revenue['Form Date'] != 'NaT']
# revenue2 = revenue2.loc[revenue2['Form Date'] != 'NaT']
revenue.dropna(subset=['Opportunity: Opportunity Name'],inplace=True)
revenue2.dropna(subset=['Opportunity: Opportunity Name'],inplace=True)

revenue2['Form Date'] = pd.to_datetime(revenue2['Form Date'], errors='coerce').dt.strftime('%Y%m%d')
revenue2['Opportunity: Created Date'] = pd.to_datetime(revenue2['Opportunity: Created Date'], errors='coerce').dt.strftime('%Y%m%d')
revenue2['Opportunity: Close Date'] = pd.to_datetime(revenue2['Opportunity: Close Date'], errors='coerce').dt.strftime('%Y%m%d')

revenue.drop_duplicates(subset = ['Opportunity: Opportunity ID'],inplace=True)
revenue2.drop_duplicates(subset = ['Opportunity: Opportunity ID'],inplace=True)

In [32]:
latest_rev = revenue.loc[(revenue['Form Created on Closed Opp'] == 0) & (revenue['Opportunity: Stage'] == 'Closed - Won')]['Opportunity: Gross Amount'].sum()
old_rev = revenue2.loc[(revenue2['Form Created on Closed Opp'] == 0) & (revenue2['Opportunity: Stage'] == 'Closed - Won')]['Opportunity: Gross Amount'].sum()
closed_projects_missing = revenue.loc[(revenue['Form Date'] >= '20210402') & (revenue['Form Date'] <= '20210405')
            & (revenue['Form Created on Closed Opp'] == 0) & (revenue['Opportunity: Stage'] == 'Closed - Won')]
merged = revenue.merge(revenue2, on='Adwire Project ID', how='outer')
merged_filtered = merged[merged['Opportunity: Gross Amount_x'] != merged['Opportunity: Gross Amount_y']]
merged_filtered = merged.loc[(merged['Form Created on Closed Opp_x'] == 0) & (merged['Opportunity: Stage_x'] == 'Closed - Won') 
                           & (merged['Opportunity: Gross Amount_x'] != merged['Opportunity: Gross Amount_y'])]
merged_filtered['gross_difference'] = merged_filtered['Opportunity: Gross Amount_x'] - merged_filtered['Opportunity: Gross Amount_y']
closed_after_april4 = revenue.loc[(revenue['Opportunity: Close Date'] >= '20210405') & (revenue['Opportunity: Close Date'] <= '20210429')
           & (revenue['Form Created on Closed Opp'] == 0) & (revenue['Opportunity: Stage'] == 'Closed - Won')]['Opportunity: Gross Amount']


print('04/29 revenue: ' + str(latest_rev))
print('04/05 revenue: ' + str(old_rev))
print('revenue difference: ' + str(latest_rev - old_rev))
print('closed betweeen 4/5 and 4/29: ' + str(closed_after_april4.sum()))
print('sum of missing projects: ' + str(closed_projects_missing['Opportunity: Gross Amount'].sum()))
print('money to account for: ' + str((latest_rev - old_rev) - (closed_after_april4.sum() -  merged_filtered.gross_difference.sum())))
print('sum of gross amount changes: ' + str(merged_filtered.gross_difference.sum()))

04/29 revenue: 28477932.51
04/05 revenue: 24714729.689999998
revenue difference: 3763202.820000004
closed betweeen 4/5 and 4/29: 2215241.76
sum of missing projects: 373622.0
money to account for: 1628328.4400000041
sum of gross amount changes: 80367.37999999999


/Users/arlisscoates/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [31]:
closed_projects_missing.to_csv('missing_closed_won.csv')